# <center>LLM - Detect AI Generated Text</center>

This competition challenges participants to develop a machine learning model that can accurately detect **whether an essay was written by a student or an LLM**. The competition dataset comprises a mix of student-written essays and essays generated by a variety of LLMs.

Team Members: 毛柏毅, 朱誼學, 許木羽, 張立誠

## Configuration

In [2]:
import transformers as T
from datasets import Dataset
import torch
# from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn as nn
import torch.nn.functional as F
import kagglehub
import numpy as np

import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display, HTML

In [3]:
from typing import Literal

HOST: Literal['Localhost', 'Interactive', 'Batch'] = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost')
IS_RERUN: bool = os.getenv('KAGGLE_IS_COMPETITION_RERUN')

print(f'HOST: {HOST}, IS_RERUN: {IS_RERUN}')

HOST: Interactive, IS_RERUN: None


In [4]:
device = torch.device(
    ("cuda:0" if torch.cuda.is_available()
     else "mps" if torch.backends.mps.is_available()
     else "cpu"))
print(device)

cuda:0


## Data

### Load Data

In [5]:
def get_kaggle_csv(dataset: str, name: str, is_comp: bool = False) -> pd.DataFrame:
    assert name.endswith('.csv')
    if IS_RERUN:
        return pd.read_csv(f'/kaggle/input/{dataset}/{name}')
    if is_comp:
        path = kagglehub.competition_download(dataset)
    else:
        path = kagglehub.dataset_download(dataset)
    return pd.read_csv(Path(path) / name)

In [6]:
if IS_RERUN:
    df_train = get_kaggle_csv('daigt-datamix', 'train_essays.csv')
    df_test = get_kaggle_csv('llm-detect-ai-generated-text', 'test_essays.csv', is_comp=True)
else:
    df_train = get_kaggle_csv('dogeon188/daigt-datamix', 'train_essays.csv')
    # split df_train into train and test
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_train.iloc[-1000:]
    df_train = df_train.iloc[:10000]

## Model

In [7]:
tokenizer = T.AutoTokenizer.from_pretrained("/kaggle/input/src/pytorch/default/1/src/tokenizer", use_fast=False)
source_classes = {'claude': 0, 'cohere': 1, 'falcon': 2, 'gpt': 3, 'llama': 4, 'mistral': 5, 'palm': 6, 'T5': 7, 'human': 8}
# Define the hyperparameters
lr = 3e-5
test_batch_size = 20

### Preprocess Data

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, split="train") -> None:
        super().__init__()
        assert split in ["train", "validation", "test"]
        if split != 'test':
            self.data = df[split]
        else:
            self.data = df
    def __getitem__(self, index):
        d = self.data.iloc[index]
        return d

    def __len__(self):
        return len(self.data)
    
def collate_fn(batch):
    texts = [item['text'] for item in batch]
    try:
        generated = [item['generated'] for item in batch]
    except:
        generated = []

    encoded_inputs = tokenizer(
        texts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=256
    )

    generated_tensor = torch.tensor(generated)

    return {
        'input_ids': encoded_inputs['input_ids'],
        'token_type_ids': encoded_inputs['token_type_ids'],
        'attention_mask': encoded_inputs['attention_mask'],
        'generated': generated_tensor
    }

### Model

In [9]:
class Deberta(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.deberta = T.AutoModel.from_pretrained("/kaggle/input/src_for_comp/pytorch/default/1/src_for_comp/deberta", num_labels=1)
        self.deberta.gradient_checkpointing_enable()

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(768, 384),
            torch.nn.Linear(384, 1)
        )
        self.activation = torch.nn.Sigmoid()
    def forward(self, **kwargs):
        input_ids = kwargs['input_ids']
        attention_mask = kwargs['attention_mask']
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

        # Mean pooling
        # reshape attention_mask, used to filter the valid tokens
        mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()  # [batch_size, seq_len, hidden_size]
        # summation for valid tokens
        sum_embeddings = torch.sum(last_hidden_state * mask, dim=1)  # [batch_size, hidden_size]
        # num of valid tokens
        sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
        x = sum_embeddings / sum_mask
        
        x = self.linear(x)
        x = self.activation(x)
        
        return {
            'generated': x
        }


## Prediction

In [10]:
model = Deberta().to(device)
model.load_state_dict(torch.load("/kaggle/input/src_for_comp/pytorch/default/1/src_for_comp/model", weights_only=True))

ds_test = CustomDataset(df_test.copy(), "test")
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=test_batch_size, collate_fn=collate_fn)

pbar = tqdm(dl_test)
pbar.set_description(f"Test")
model.eval()

final_preds = []

with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            generated = batch['generated'].to(device)
            
            pred = model(input_ids=input_ids, attention_mask=attention_mask)
            
            final_preds.extend(list(map(lambda x: x.item(), pred['generated'])))

Test: 100%|██████████| 50/50 [00:21<00:00,  2.37it/s]


## Evaluation

In [11]:
# validation
if not IS_RERUN:
    from sklearn.metrics import roc_auc_score

    auc_score = roc_auc_score(df_test['generated'], final_preds)
    
    print(f"ROC AUC: {auc_score:.4f}")

ROC AUC: 0.4949


## Submission

In [12]:
df_test['generated'] = final_preds
submission = df_test[['id' if IS_RERUN else 'prompt_id', 'generated']]
submission.to_csv('submission.csv', index=False)